In [1]:
from pysmt.shortcuts import Symbol, And, GE, LE, Int, Solver, Equals, Implies, Or, NotEquals, Not, Iff, Plus, Ite, LT, ExactlyOne, ToReal
from pysmt.logics import QF_LIA, QF_LRA
from pysmt.typing import INT
from pysmt.exceptions import SolverReturnedUnknownResultError
from pebble import concurrent
from concurrent.futures import TimeoutError, ThreadPoolExecutor, as_completed
from utils import *
from SMT_utils import *
import time as t

In [2]:
##### Leggiamo un'istanza ####
instance_num = 1

instance = load_instance(dir="../input", instance_num=instance_num)

In [3]:
exactly_one = ExactlyOne

In [4]:
def set_constraints(instance, solver):
    # prendiamo i dati in ingresso dell'istanza
    m, n, l, s, D = instance.get_values()


    # DECISION VARIABLES

    # X_i_k = j  <->  il corriere i è in posizione j al momento k
    # i = 0,..., m-1 (corrieri)
    # k = 0,..., n+1 (momenti temporali)
    # j = 1,..., n+1 dove n+1 è la base (posizione)
    # X = [[Symbol(f"x_{i}_{k}", INT) for i in range(m)] for k in range(n+2)]
    X = [[Symbol(f"x_{i}_{k}", INT) for k in range(n+2)] for i in range(m)]


    # carried_load[i] terrà il peso totale portato dal corriere i-esimo
    carried_load = [Symbol(f"carried_load_{i}", INT) for i in range(m)]

    # traveled_distance[i] terrà la distanza totale percorsa dal corriere i-esimo
    traveled_distance = [Symbol(f"traveled_distance_{i}", INT) for i in range(m)]

    # obj_function sarà il valore massimo tra tutte le distanze percorse
    obj_function = Symbol(f"obj_function", INT)
    

    # definiamo il dominio di X, j = 1,..., n+1 dove n+1 è la base (posizione)
    domain_X = []
    for row in X:
        row_domain = And([And(GE(x, Int(1)),
                                LE(x, Int(n+1))) for x in row])
        domain_X.append(row_domain)

    # imponiamo il dominio di X al solver
    solver.add_assertion(And(domain_X))



    # TUTTI I CORRIERI DEVONO INIZIARE IN BASE (al momento k=0 il corriere è in j=n+1)
    for i in range(m):
        solver.add_assertion(Equals(X[i][0], Int(n+1)))


    # TUTTI I CORRIERI PRIMA O POI DEVONO TORNARE IN BASE (in un momento k>=1 il corriere dev'essere almeno una volta in j=n+1)
    for i in range(m):
        back_row_constr = Or(
            [Equals(X[i][k], Int(n+1)) for k in range(1, n+2)]
        )
        solver.add_assertion(back_row_constr)
   
    
    # SE UN CORRIERE È IN BASE AL MOMENTO K ALLORA LO SARÀ ANCHE AL MOMENTO K+1 (esluso il momento 0)
    for i in range(m):
        for k in range(1, n+1):
            solver.add_assertion(
                                Implies(
                                    Equals(X[i][k], Int(n+1)),
                                    Equals(X[i][k+1], Int(n+1))
                                )
            )

    
    # TUTTI I LUOGHI DEVONO ESSERE VISITATI SOLO UNA VOLTA
    for j in range(1, n+1):
        assertions = [Equals(X[i][k], Int(j)) for i in range(m) for k in range(1, n + 1)]
        # solver.add_assertion(exactly_one(assertions, f"one_visit_{j}")) # <------ usa questo per il tuo exactly_one
        solver.add_assertion(exactly_one(assertions)) # <----- usa questo per l'ExactlyOne nativo di pysmt
    

    
    # carried_load[i] avrà la somma di tutti i pesi trasportati dal corrieri i
    for i in range(m):
        # In implications metto le implicazioni che uso per calcolare le somme dei pesi
        implications = []
        for j in range(1, n+1):
            # Metto qui la somma per ogni posizione visitata
            # Se X[i][k] è uguale a j allora in Plus metto s[j-1], altrimenti metto 0
            # In sum_expr metto la somma di tutti i pesi trasportati
            sum_expr = Plus([Ite(Equals(X[i][k], Int(j)),
                                 Int(s[j - 1]), 
                                 Int(0)) for k in range(1, n+1)])
            
            implications.append(sum_expr)

        # Aggiungi l'assertion al solver: carried_load[i] = somma dei pesi trasportati
        solver.add_assertion(Equals(carried_load[i], Plus(implications)))



    # OGNI CORRIERE NON PUÒ TRASPORTARE PIÙ DI UN CERTO PESO
    for i in range(m):
        solver.add_assertion(LE(carried_load[i], Int(l[i])))


    
    # traveled_distances[i] avrà la somma delle distanza percorse dal corriere i
    for i in range(m):
        distances = []

        # prendo la distanza dalla posizione iniziale
        for j in range(1, n+2):
            first_distance = Plus([Ite(Equals(X[i][1], Int(j)),
                                        Int(int(D[n][j-1])),
                                        Int(0))])
            distances.append(first_distance)
            
        # prendo le distanze tra due posizioni visitate consecutivamente
        for k in range(1, n+1):

            for j1 in range(1, n + 2):
                for j2 in range(1, n + 2):
                    distances.append(Plus([Ite(And(Equals(X[i][k], Int(j1)),
                                                Equals(X[i][k+1], Int(j2))),
                                                Int(int(D[j1-1][j2-1])),
                                                Int(0)
                                            )]))

        solver.add_assertion(Equals(traveled_distance[i], Plus(distances)))


    # impongo che la obj_function sia il valore maggiore tra le distanze percorse dai corrieri
    for i in range(m):
        solver.add_assertion(GE(obj_function, traveled_distance[i]))


    # return variables
    return X, carried_load, traveled_distance, obj_function

In [5]:
def set_solver(timeout, solver_name):
    # print(timeout)
    if solver_name == 'z3':
        return {'timeout': timeout} 
    elif solver_name == 'cvc5':
        return {'tlimit': str(timeout)}  
    else:
        return {} 

In [9]:
def linear_optimization(instance, solver, timeout, X, carried_load, traveled_distance, obj_function):

    m, n, l, s, D = instance.get_values()

    previousModel = None
    is_sat = True
    is_optimal = True

    # Start of the counter
    start_time = t.time()
    # This is a flag used to indicate the i-th found solution
    solution_number = 0
    

    solver.push()

    while(is_sat):
        # prendo lo stato dal solver
        status = solver.check_sat()

        if status is True:
            # ho trovato una soluzione quindi incremento il counter
            solution_number = solution_number + 1
            # print(solution_number)

            # salvo questo modello come previousModel
            previousModel = solver.get_model()

            # calcolo quanto tempo è passato
            current_time = t.time()
            passed_time = int((current_time - start_time))

            # setto il timeout per il solver
            # set_solver((timeout - passed_time)*1000, solver)
            set_solver((timeout - passed_time), solver)

            # prendo la objective function dal modello
            previous_obj_function = previousModel.get_value(obj_function)
            print(previous_obj_function)
            # print(type(previous_obj_function))

            # impongo al solver di trovare una soluzione migliore
            solver.add_assertion(LT(obj_function, previous_obj_function))

        elif status is False:
            # se il counter di soluzioni è a zero vuol dire che non ho mai trovato almeno una soluzione
            if solution_number == 0:
                print(status)
                current_time = t.time()
                passed_time = int((current_time - start_time))
                return passed_time, False, "N/A unsat", []
             # esco dal ciclo di ricerca delle soluzioni
            is_sat = False

        else:
            # se il counter di soluzioni è a zero vuol dire che non ha avuto il tempo di trovarla
            if solution_number == 0:
                print("TIME EXCEEDED")
                return timeout, False, "N/A unknown", []
            
            # esco dal ciclo di ricerca delle soluzioni
            is_sat = False
            is_optimal = False

    current_time = t.time()
    passed_time = current_time - start_time

    # come modello riprendo l'ultimo trovato
    model = previousModel

    '''
    # prendo dal modello la soluzione trovata
    for i in range(instance.m):
        print(f"Percorso del corriere numero {i}:")
        for k in X[i]:
            print(f"{k}: {model.get_value(k)}")
        print(f"Peso portato dal corriere numero {i}:")
        print(f"{model.get_value(carried_load[i])}")
        print(f"Distanza totale percorsa dal corriere {i}:")
        print(f"{model.get_value(traveled_distance[i])}")
    print(f"Quindi, la funzione obiettivo è: {model.get_value(obj_function)}")
    '''
    
    visited_locations = [[] for _ in range(m)]
    for i in range(m):
        for k in X[i]:
            sol_assignment = model.get_value(k)
            if sol_assignment != Int(n+1):
                visited_locations[i].append(sol_assignment)
    
    return int(passed_time), is_optimal, model.get_value(obj_function), visited_locations


In [7]:


def binary_optimization(instance, solver, timeout, X, carried_load, traveled_distance, obj_function):

    m, n, l, s, D = instance.get_values()
    
    previousModel = None
    is_sat = True
    is_optimal = True

    # We set upper and lower bound for the objective function
    lower_bound = set_lower_bound(D)
    upper_bound = set_upper_bound(D, n)

    # Start of the counter
    start_time = t.time()
    # This is a flag used to indicate the i-th found solution
    solution_number = 0

    solver.push()
    
    while(is_sat):

        # If upper_bound - lower_bound > 1 then we set the middle_bound in the middle of those
        if (upper_bound - lower_bound > 1):
            middle_bound = int(np.ceil((upper_bound + lower_bound) / 2))
        # If upper_bound - lower_bound == 1 we cannot devide the search space in two: we choose as middle bound the smaller bound
        elif (upper_bound - lower_bound == 1):
            middle_bound = int(lower_bound)
            is_sat = False
        elif (upper_bound - lower_bound == 0):
            middle_bound = int(lower_bound)
            is_sat = False
                
        if (upper_bound - lower_bound < 0):
            is_sat = False

        print(f"Middle bound is: {middle_bound}")


        # We impose the solver to find an objective function smaller than the middle bound  (a better solution)
        solver.add_assertion(LE(obj_function, Int(middle_bound)))

        # We set the time for next solution
        current_time = t.time()
        passed_time = int(current_time - start_time)
        # set_solver((timeout - passed_time)*1000, solver)
        set_solver((timeout - passed_time), solver)

        
        # prendo lo stato dal solver
        status = solver.check_sat()


        if status is True:
            solution_number = solution_number + 1
            print(f"Solution number: {solution_number}, status: {status}")
            print()


            # salvo questo modello come previousModel
            previousModel = solver.get_model()

            # We set the time for next solution
            current_time = t.time()
            passed_time = int(current_time - start_time)
            # set_solver((timeout - passed_time)*1000, solver)
            set_solver((timeout - passed_time), solver)

            # prendo la objective function dal modello
            previous_obj_function = previousModel.get_value(obj_function)

            # update dell'upper_bound
            upper_bound = previous_obj_function.constant_value()
            print(f"il nuovo upper bound è: {upper_bound}")
            
        elif status is False:
            print(status)
            print()
            solution_number = solution_number + 1


            # We delete the last solver (useless solver)
            solver.pop()
            # We retake the previous one
            solver.push()
            # We set the lower bound as the middle because we need to search in the second part of the search space
            lower_bound = int(middle_bound)

        else:
            if solution_number == 0:
                return timeout, False, "N/A", []
            
            is_sat = False
            is_optimal = False


    current_time = t.time()
    passed_time = current_time - start_time

    # come modello riprendo l'ultimo trovato
    model = previousModel

    '''
    # prendo dal modello la soluzione trovata
    for i in range(instance.m):
        print(f"Percorso del corriere numero {i}:")
        for k in X[i]:
            print(f"{k}: {model.get_value(k)}")
        print(f"Peso portato dal corriere numero {i}:")
        print(f"{model.get_value(carried_load[i])}")
        print(f"Distanza totale percorsa dal corriere {i}:")
        print(f"{model.get_value(traveled_distance[i])}")
    print(f"Quindi, la funzione obiettivo è: {model.get_value(obj_function)}")
    '''
    
    visited_locations = [[] for _ in range(m)]
    for i in range(m):
        for k in X[i]:
            sol_assignment = model.get_value(k)
            if sol_assignment != Int(n+1):
                visited_locations[i].append(sol_assignment)
    
    return int(passed_time), is_optimal, model.get_value(obj_function), visited_locations
        
    

In [8]:
timeout = 300 * 1000
# solvers = ['z3', 'cvc5']
solvers = ['z3']
for solver_name in solvers:
    with Solver(name = solver_name, solver_options = set_solver(timeout, solver_name), logic = QF_LIA) as solver:
        '''
        X, carried_load, traveled_distance, obj_function = set_constraints(instance, solver)
        time, optimal, obj, sol = binary_optimization(instance, solver, timeout, X, carried_load, traveled_distance, obj_function)
    print(f"The solver took {time} seconds to solve the instance")
    print(f"The optimal flag is: {optimal}")
    print(f"The objective function is: {obj}")
    print(f"The solution is: {sol}")
        '''

        X, carried_load, traveled_distance, obj_function = set_constraints(instance, solver)

        '''
        found_sol = solver.solve()
        print(solver.check_sat())

        if found_sol:
            print("#### Soluzione trovata #####\n")
            # prendo tutte le variabili x_i_k
            for i in range(instance.m):
                
                print(f"Percorso del corriere numero {i}:")
                for k in X[i]:
                    print(f"{k}: {solver.get_value(k)}")
                print(f"Peso portato dal corriere numero {i}:")
                print(f"{solver.get_value(carried_load[i])}")
                print(f"Distanza totale percorsa dal corriere {i}:")
                print(f"{solver.get_value(traveled_distance[i])}")
                
            print(f"Quindi, la funzione obiettivo è: {solver.get_value(obj_function)}")
        else:
            print("Nessuna soluzione trovata.")
    
        '''

        while(solver.check_sat()):
            try:
                previous_obj_function = int(solver.get_value(obj_function).serialize())
                solver.add_assertion(LT(obj_function, Int(previous_obj_function)))

                found_sol = solver.solve()
                print(solver.check_sat())
                print(f"Quindi, la funzione obiettivo è: {solver.get_value(obj_function)}")
                
            except SolverReturnedUnknownResultError as e:
                print("Non avevo tempo!")
                break


True
Quindi, la funzione obiettivo è: 732
True
Quindi, la funzione obiettivo è: 731
True
Quindi, la funzione obiettivo è: 730
True
Quindi, la funzione obiettivo è: 729
True
Quindi, la funzione obiettivo è: 728
True
Quindi, la funzione obiettivo è: 727
True
Quindi, la funzione obiettivo è: 726
True
Quindi, la funzione obiettivo è: 725
True
Quindi, la funzione obiettivo è: 724
True
Quindi, la funzione obiettivo è: 723
True
Quindi, la funzione obiettivo è: 722
True
Quindi, la funzione obiettivo è: 721
True
Quindi, la funzione obiettivo è: 720
True
Quindi, la funzione obiettivo è: 719
True
Quindi, la funzione obiettivo è: 718
True
Quindi, la funzione obiettivo è: 717
True
Quindi, la funzione obiettivo è: 716
True
Quindi, la funzione obiettivo è: 715
True
Quindi, la funzione obiettivo è: 714
True
Quindi, la funzione obiettivo è: 713
True
Quindi, la funzione obiettivo è: 712
True
Quindi, la funzione obiettivo è: 711
True
Quindi, la funzione obiettivo è: 710
True
Quindi, la funzione obiettivo

In [6]:
timeout = 30 * 1000
solvers = ['cvc5']
LAST_VALUE = None

@concurrent.process(timeout = 30)
def solve(solver_name, timeout):
    # for solver_name in solvers:
        print(f"### SOLVING WITH {solver_name} AS SOLVER ###")
        with Solver(name = solver_name, solver_options = set_solver(timeout, solver_name), logic = QF_LIA) as solver:

            X, carried_load, traveled_distance, obj_function = set_constraints(instance, solver)

            last_value = None
            while(solver.check_sat()):
                try:
                    previous_obj_function = int(solver.get_value(obj_function).serialize())
                    solver.add_assertion(LT(obj_function, Int(previous_obj_function)))

                    found_sol = solver.solve()
                    print(solver.check_sat())
                    # print(f"Quindi, la funzione obiettivo è: {solver.get_value(obj_function)}")
                    LAST_VALUE = solver.get_value(obj_function)
                    last_value = solver.get_value(obj_function)
                    
                except SolverReturnedUnknownResultError as e:
                    print("Non avevo tempo!")
                    break
            
            return last_value

In [ ]:
try:
    for solver_name in solvers:
        print(solve(solver_name, timeout = 300 * 1000))
        
        print(f"The best value is: {LAST_VALUE}")

except TimeoutError:
    print('TIMEOUT!')

<ProcessFuture at 0x7f1a0e57c310 state=running>
The best value is: None


### SOLVING WITH cvc5 AS SOLVER ###


True
True
True
True
True
True
True


In [ ]:

# @concurrent.process(timeout = 30)
def solve_with_pebble(solver_name, timeout):
    # for solver_name in solvers:
        print(f"### SOLVING WITH {solver_name} AS SOLVER ###")
        with Solver(name = solver_name, solver_options = set_solver(timeout, solver_name), logic = QF_LIA) as solver:

            X, carried_load, traveled_distance, obj_function = set_constraints(instance, solver)

            last_value = None
            while(solver.check_sat()):
                try:
                    previous_obj_function = int(solver.get_value(obj_function).serialize())
                    solver.add_assertion(LT(obj_function, Int(previous_obj_function)))

                    found_sol = solver.solve()
                    # print(solver.check_sat())
                    # print(f"Quindi, la funzione obiettivo è: {solver.get_value(obj_function)}")
                    last_value = solver.get_value(obj_function)
                    
                except SolverReturnedUnknownResultError as e:
                    print("Non avevo tempo!")
                    break
            
            return last_value




def execute_solver(solver_name, timeout):
        with ThreadPoolExecutor(max_workers=1) as executor:
            future = executor.submit(solve_with_pebble, solver_name, timeout)

            # if future.done() == True:
            last_value = future.result(timeout=5)  # Attende che la funzione completi e ottiene il risultato
            print(f"The best value is: {last_value}")


timeout = 300 * 1000
execute_solver('z3', timeout)

### SOLVING WITH z3 AS SOLVER ###
Future ha finito!
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True
True


In [ ]:
######################## TENTATIVO UTILIZZANDO SUBPROCESSES ########################
import multiprocessing


def solve_with_multiprocessing(shared_obj_value, solver_name):
        print(f"### SOLVING WITH {solver_name} AS SOLVER ###")
        with Solver(name = solver_name, logic = QF_LIA) as solver:

            X, carried_load, traveled_distance, obj_function = set_constraints(instance, solver)

            while(solver.check_sat()):
                try:
                    previous_obj_function = int(solver.get_value(obj_function).serialize())
                    solver.add_assertion(LT(obj_function, Int(previous_obj_function)))

                    found_sol = solver.solve()
                    # print(solver.check_sat())
                    print(f"Quindi, la funzione obiettivo è: {previous_obj_function}")
                    shared_obj_value.value = previous_obj_function


                    
                except SolverReturnedUnknownResultError as e:
                    print("Non avevo tempo!")
                    break
        


def run_solve_with_timeout(func, multiprocessing_timeout):
    # Crea un valore condiviso
    best_obj_value = multiprocessing.Value('i', 0)  # 'i' indica un intero, inizialmente è a zero

    solver_name = 'cvc5'
    tim = 300 * 1000

    # Crea un processo per eseguire la funzione e lo fa partire
    process = multiprocessing.Process(target=func, args=(best_obj_value, solver_name, tim))
    process.start()

    # Aspetta che il processo termini o che scada il timeout
    process.join(multiprocessing_timeout)

    # se dopo il tempo di timeout il processo è ancora vivo allora lo killo
    if process.is_alive():
        print("Timeout! Terminando il processo.")
        process.terminate()  # Termina il processo se è ancora attivo
        process.join()  # Aspetta che il processo termini
    else:
        print("Processo completato.")

    # Stampa il valore finale
    print("Ultimo valore trovato per la objective function:", best_obj_value.value)



run_solve_with_timeout(solve_with_multiprocessing, multiprocessing_timeout=30)

### SOLVING WITH cvc5 AS SOLVER ###
Quindi, la funzione obiettivo è: 728
Quindi, la funzione obiettivo è: 727
Quindi, la funzione obiettivo è: 726
Quindi, la funzione obiettivo è: 725
Quindi, la funzione obiettivo è: 724
Quindi, la funzione obiettivo è: 723
Quindi, la funzione obiettivo è: 722
Quindi, la funzione obiettivo è: 721
Quindi, la funzione obiettivo è: 720
Quindi, la funzione obiettivo è: 719
Quindi, la funzione obiettivo è: 718
Quindi, la funzione obiettivo è: 717
Quindi, la funzione obiettivo è: 716
Quindi, la funzione obiettivo è: 715
Quindi, la funzione obiettivo è: 714
Quindi, la funzione obiettivo è: 713
Quindi, la funzione obiettivo è: 712
Quindi, la funzione obiettivo è: 711
Quindi, la funzione obiettivo è: 710
Quindi, la funzione obiettivo è: 709
Quindi, la funzione obiettivo è: 708
Quindi, la funzione obiettivo è: 707
Quindi, la funzione obiettivo è: 706
Quindi, la funzione obiettivo è: 705
Quindi, la funzione obiettivo è: 704
Quindi, la funzione obiettivo è: 703
Qu

In [5]:
import multiprocessing

def create_PYSMT_solver(solver_name, logic):
    with Solver(name = solver_name, logic = logic) as solver:
        return solver


def set_visited_locations(instance, X, model, shared_visited_locations):
    m, n, l, s, D = instance.get_values()

    # creo il percorso del corriere i-esimo
    # print("la funzione è stata chiamata")
    for i in range(m):
        courier_path = []
        for k in X[i]:
            sol_assignment = model.get_value(k)
            if sol_assignment != Int(n+1):
                courier_path.append(sol_assignment)
                
        # print(courier_path)
        shared_visited_locations[i] = courier_path


def LO(instance, X, solver, obj_function, shared_obj_function, shared_is_optimal, shared_visited_locations):

    previousModel = None
    is_sat = True

    # This is a flag used to indicate the i-th found solution
    solution_number = 0

    solver.push()

    while(is_sat):
        # prendo lo stato dal solver
        status = solver.check_sat()

        if status is True:
            # ho trovato una soluzione quindi incremento il counter
            solution_number = solution_number + 1
            # print(solution_number)

            # salvo questo modello come previousModel
            previousModel = solver.get_model()

            # prendo la objective function dal modello
            previous_obj_function = int(solver.get_value(obj_function).serialize())
            # print(previous_obj_function)
            # print(type(previous_obj_function))

            # impongo al solver di trovare una soluzione migliore
            solver.add_assertion(LT(obj_function, Int(previous_obj_function)))

            # aggiorno il valore condiviso tra i processi
            shared_obj_function.value = previous_obj_function
            set_visited_locations(instance, X, previousModel, shared_visited_locations)


        elif status is False:
            # se il counter di soluzioni è a zero vuol dire che non ho mai trovato almeno una soluzione
            if solution_number == 0:
                print(status)
                return False, "N/A unsat", []
             # esco dal ciclo di ricerca delle soluzioni
            is_sat = False

        else:
            # se il counter di soluzioni è a zero vuol dire che non ha avuto il tempo di trovarla
            if solution_number == 0:
                print("TIME EXCEEDED")
                return timeout, False, "N/A unknown", []
            
            # esco dal ciclo di ricerca delle soluzioni
            is_sat = False
            shared_is_optimal.value = False

    # se sono qui ho avuto il tempo di trovare la soluzione migliore possibile
    shared_is_optimal.value = True
    # come modello riprendo l'ultimo trovato
    model = previousModel
    # aggiorno il valore finale della objective function condivisa tra i processi
    shared_obj_function.value = int(model.get_value(obj_function).serialize())



def solve_multiprocessing(optimization_flag, timeout):
    start_time = t.time()

    # creo il solver PYSMT
    with Solver(name = 'cvc5', logic = QF_LIA) as solver:

        # creo i valori condivisi per i multiprocessi
        s_obj_function = multiprocessing.Value('i', 0)  # 'i' indica un intero, inizialmente è a zero
        s_is_optimal = multiprocessing.Value('b', False) # 'b' indica che è un booleano, lo inizializzo a False
        s_visited_locations = multiprocessing.Manager().dict() # uso un dizionario per i percorsi dei corrieri


        # setto i contraints del solver
        X, carried_load, traveled_distance, obj_function = set_constraints(instance, solver)


        if optimization_flag == "linear":
            # creo il processo usando come ottimizzatore quello lineare
            process = multiprocessing.Process(target=LO, args=(instance, X, solver, obj_function, s_obj_function, s_is_optimal, s_visited_locations))


        
        # faccio partire il processo con l'ottimizzatore
        process.start()

        # Aspetta che il processo termini o che scada il timeout, dal timeout devo torgliere il tempo impiegato per creare il solver 
        current_time = t.time()
        passed_time = int((current_time - start_time))
        process_timeout = timeout - passed_time
        process.join(process_timeout)

        # se dopo il tempo di timeout il processo è ancora vivo allora lo killo
        if process.is_alive():
            print("Timeout! Terminando l'ottimizzatore.")
            process.terminate()  # Termina il processo se è ancora attivo
            process.join()  # Aspetta che il processo termini
        else:
            print("Processo completato.")


    # Prendo i valori che mi servono: tempo impiegato dall'ottimizzatore, ultima objective function trovata, ottimalità e le visited_locations
    # visited_locations = get_visited_locations(instance, X, solver)
    current_time = t.time()
    passed_time = current_time - start_time


    return int(passed_time), s_is_optimal.value, s_obj_function.value, s_visited_locations


def from_dict_to_list(dictionary):
    m = len(dictionary)
    solution = [[] for _ in range(m)]

    for key, value in dictionary.items():
        solution[key] = value

    return solution


passed_time, s_is_optimal, s_obj_function, s_visited_locations = solve_multiprocessing("linear", 20)
sol = from_dict_to_list(s_visited_locations)

if s_is_optimal == 1:
    optimality = True
else:
    optimality = False

print(f"The solver took {passed_time} seconds to solve the instance")
print(f"The optimal flag is: {optimality}")
print(f"The objective function is: {s_obj_function}")
print(f"The solution is: {sol}")


Processo completato.
The solver took 0 seconds to solve the instance
The optimal flag is: True
The objective function is: 14
The solution is: [[4, 3, 1], [2, 5, 6]]
